In [1]:
!pip install imageio-ffmpeg

In [2]:
import os
import sys
import logging
import tempfile
import shutil
import subprocess
import traceback
import gc
from typing import List, Set, Dict, Optional, Any
from dataclasses import dataclass, field
from urllib.parse import urlparse
from concurrent.futures import ThreadPoolExecutor, as_completed
from contextlib import contextmanager
from pathlib import Path

In [3]:
# Import torch first to avoid registration conflicts
import torch
import torchvision  # Import this explicitly before transformers

# Env setup
import os
os.environ.setdefault("TRANSFORMERS_CACHE", "/tmp/transformers_cache")
os.environ.setdefault("HF_HOME", "/tmp/hf_home")
os.environ.setdefault("TORCH_HOME", "/tmp/torch_home")

import boto3
from botocore.exceptions import ClientError
import pandas as pd
from tqdm import tqdm
import imageio_ffmpeg
from transformers import pipeline

/opt/conda/lib/python3.12/site-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2025-10-05 20:12:35.609523: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(levelname)s - %(message)s",
    handlers=[logging.StreamHandler(sys.stdout)]
)
logger = logging.getLogger("asr_pipeline")

In [5]:
# =============================================== Config =============================================== 
@dataclass
class Config:
    """Configuration for the transcription pipeline"""
    # S3 settings
    s3_input: str = "s3://asrelder-data/common_voice/clips/"
    output_local_csv: str = "./transcripts_from_prefix.csv"
    write_back_to_s3: bool = False
    output_s3_uri: Optional[str] = None 
    #output_s3_uri: str = "s3://asrelder-data/outputs/transcripts_from_prefix.csv" uncomment if we want it back on s3
    
    # Processing settings
    max_files: Optional[int] = 50
    download_workers: int = 4
    append_every_n: int = 20
    resume_from_csv: bool = True
    
    # Model settings
    model_id: str = "openai/whisper-base"
    language: Optional[str] = None  # change to en
    task: str = "transcribe"
    chunk_length_s: int = 30
    stride_length_s: tuple = (5, 5)
    
    # File settings
    audio_extensions: List[str] = field(default_factory=lambda: [".mp3", ".wav", ".flac", ".m4a", ".ogg"])

In [6]:
# =============================================== Core Components =============================================== 
class FFmpegSetup:
    """Manages FFmpeg availability"""
    
    @staticmethod
    def ensure_available() -> Optional[str]:
        """Check if ffmpeg is available on PATH"""
        ff = None
        try:
            ff = imageio_ffmpeg.get_ffmpeg_exe()
        except Exception as e:
            logger.warning(f"imageio-ffmpeg error: {e}")
        
        if ff and os.path.exists(ff):
            ff_dir = os.path.dirname(ff)
            os.environ["PATH"] = ff_dir + os.pathsep + os.environ.get("PATH", "")
        
        resolved = shutil.which("ffmpeg")
        if resolved:
            try:
                out = subprocess.run(
                    [resolved, "-version"],
                    stdout=subprocess.PIPE,
                    stderr=subprocess.STDOUT,
                    check=True,
                    timeout=5
                )
                logger.info(f"ffmpeg: {resolved} | {out.stdout.decode('utf-8').splitlines()[0]}")
            except Exception:
                logger.info(f"ffmpeg: {resolved} (version check failed)")
        else:
            # Create shim if needed
            if ff and os.path.exists(ff):
                bin_dir = os.path.expanduser("~/.local/bin")
                os.makedirs(bin_dir, exist_ok=True)
                shim = os.path.join(bin_dir, "ffmpeg")
                with open(shim, "w") as f:
                    f.write(f"#!/usr/bin/env bash\n\"{ff}\" \"$@\"\n")
                os.chmod(shim, 0o755)
                os.environ["PATH"] = bin_dir + os.pathsep + os.environ.get("PATH", "")
                resolved = shutil.which("ffmpeg")
                if resolved:
                    logger.info(f"ffmpeg shim created: {resolved}")
        
        if not resolved:
            logger.warning("FFmpeg not found; use torchaudio fallback")
        
        return resolved

class S3Manager:
    """Handles S3 operations"""
    
    def __init__(self, config: Config):
        self.config = config
        self.client = boto3.client("s3")
    
    def parse_uri(self, uri: str) -> tuple[str, str]:
        """Parse S3 URI into bucket and key"""
        if not uri.startswith("s3://"):
            raise ValueError(f"Invalid S3 URI: {uri}")
        p = urlparse(uri)
        return p.netloc, p.path.lstrip("/")
    
    def is_audio_file(self, key: str) -> bool:
        """Check if key is an audio file"""
        return any(key.lower().endswith(ext) for ext in self.config.audio_extensions)
    
    def list_audio_keys(self, bucket: str, prefix: str) -> List[str]:
        """List all audio keys under prefix"""
        if self.is_audio_file(prefix):
            return [prefix]
        
        keys = []
        paginator = self.client.get_paginator("list_objects_v2")
        
        for page in paginator.paginate(Bucket=bucket, Prefix=prefix):
            for obj in page.get("Contents", []):
                key = obj["Key"]
                if not key.endswith("/") and self.is_audio_file(key):
                    keys.append(key)
                    if self.config.max_files and len(keys) >= self.config.max_files:
                        return keys
        return keys
    
    def download_to_temp(self, bucket: str, key: str) -> str:
        """Download S3 object to temporary file"""
        _, ext = os.path.splitext(key)
        if not ext:
            ext = ".mp3"
        
        fd, tmp_path = tempfile.mkstemp(suffix=ext)
        os.close(fd)
        
        with open(tmp_path, "wb") as f:
            self.client.download_fileobj(bucket, key, f)
        
        return tmp_path
    
    def upload_file(self, local_path: str, s3_uri: str):
        """Upload file to S3"""
        bucket, key = self.parse_uri(s3_uri)
        self.client.upload_file(local_path, bucket, key)
        logger.info(f"Uploaded to {s3_uri}")

class TranscriptionManager:
    """Manages the ASR pipeline and transcription"""
    
    def __init__(self, config: Config):
        self.config = config
        self.pipeline = self._build_pipeline()
    
    def _build_pipeline(self):
        """Build the ASR pipeline"""
        use_cuda = torch.cuda.is_available()
        device = 0 if use_cuda else -1
        dtype = torch.float16 if use_cuda else torch.float32
        
        generate_kwargs = {}
        if self.config.language:
            generate_kwargs["language"] = self.config.language
        if self.config.task:
            generate_kwargs["task"] = self.config.task
        
        logger.info(f"Loading ASR: {self.config.model_id} (device={'cuda' if use_cuda else 'cpu'}, dtype={dtype})")
        
        return pipeline(
            "automatic-speech-recognition",
            model=self.config.model_id,
            device=device,
            torch_dtype=dtype,
            return_timestamps=True,
            chunk_length_s=self.config.chunk_length_s,
            stride_length_s=self.config.stride_length_s,
            generate_kwargs=generate_kwargs or None,
        )
    
    def transcribe(self, audio_path: str) -> Dict[str, Any]:
        """Transcribe audio file"""
        try:
            result = self.pipeline(audio_path)
            text = result.get("text", "") if isinstance(result, dict) else str(result)
            return {"text": text, "error": None}
        except Exception as e:
            if "ffmpeg" in str(e).lower():
                return self._fallback_transcribe(audio_path)
            return {"text": "", "error": f"{type(e).__name__}: {e}"}
    
    def _fallback_transcribe(self, audio_path: str) -> Dict[str, Any]:
        """Fallback using torchaudio"""
        try:
            import torchaudio
            waveform, sr = torchaudio.load(audio_path)
            if waveform.ndim == 2:
                waveform = waveform.mean(dim=0, keepdim=True)
            
            result = self.pipeline(waveform.squeeze(0).numpy(), sampling_rate=sr)
            text = result.get("text", "") if isinstance(result, dict) else str(result)
            return {"text": text, "error": None}
        except Exception as e:
            return {"text": "", "error": f"Fallback failed: {e}"}
    
    def cleanup_gpu_memory(self):
        """Clean up GPU memory"""
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        gc.collect()

class CSVManager:
    """Handles CSV operations """
    
    def __init__(self, csv_path: str):
        self.csv_path = csv_path
    
    def read_processed_keys(self) -> Set[str]:
        """Read already processed S3 keys from CSV"""
        if not os.path.exists(self.csv_path):
            return set()
        try:
            df = pd.read_csv(self.csv_path, usecols=["s3_key"])
            return set(df["s3_key"].astype(str).tolist())
        except Exception as e:
            logger.warning(f"Could not read existing CSV: {e}")
            return set()
    
    def append_results(self, results: List[Dict[str, Any]]):
        """Append results to CSV"""
        if not results:
            return
        
        df = pd.DataFrame(results)
        mode = "a" if os.path.exists(self.csv_path) else "w"
        header = not os.path.exists(self.csv_path)
        
        df.to_csv(self.csv_path, index=False, mode=mode, header=header)
        logger.info(f"Appended {len(results)} rows to {self.csv_path}")

In [7]:
# ================================= Main Pipeline ===========================================

class ProductionPipeline:
    """Main for the transcription pipeline"""
    
    def __init__(self, config: Config):
        self.config = config
        self.s3_manager = S3Manager(config)
        self.transcription_manager = TranscriptionManager(config)
        self.csv_manager = CSVManager(config.output_local_csv)
        self.results_buffer: List[Dict[str, Any]] = []
    
    def run(self):
        """Execute the transcription pipeline"""
        # Setup
        FFmpegSetup.ensure_available()
        
        # Get files to process
        bucket, prefix = self.s3_manager.parse_uri(self.config.s3_input)
        all_keys = self.s3_manager.list_audio_keys(bucket, prefix)
        
        processed_keys = set()
        if self.config.resume_from_csv:
            processed_keys = self.csv_manager.read_processed_keys()
        
        keys_to_process = [k for k in all_keys if k not in processed_keys]
        
        logger.info(
            f"Processing {len(keys_to_process)} files "
            f"(skipped {len(processed_keys)} already done)"
        )
        
        if not keys_to_process:
            logger.info("No files to process")
            return
        
        # Process with concurrent downloads
        self._process_with_concurrency(bucket, keys_to_process)
        
        # Final flush
        self._flush_results()
        
        # Upload to S3 if configured
        if self.config.write_back_to_s3:
            self.s3_manager.upload_file(
                self.config.output_local_csv,
                self.config.output_s3_uri
            )
        
        logger.info("Pipeline complete!")
    
    def _process_with_concurrency(self, bucket: str, keys: List[str]):
        """Process files with concurrent downloads"""
        batch_size = max(1, self.config.download_workers * 2)
        
        with ThreadPoolExecutor(max_workers=self.config.download_workers) as pool:
            for i in range(0, len(keys), batch_size):
                batch = keys[i:i + batch_size]
                futures = {
                    pool.submit(self.s3_manager.download_to_temp, bucket, k): k
                    for k in batch
                }
                
                progress = tqdm(
                    as_completed(futures),
                    total=len(futures),
                    desc=f"Batch {i//batch_size + 1}"
                )
                
                for future in progress:
                    key = futures[future]
                    self._process_single_file(future, key)
    
    def _process_single_file(self, future, key: str):
        """Process a single downloaded file"""
        local_path = None
        try:
            # Get downloaded file
            local_path = future.result()
            
            # Transcribe
            result = self.transcription_manager.transcribe(local_path)
            
            # Store result
            self.results_buffer.append({
                "s3_key": key,
                "filename": os.path.basename(key),
                "transcribed_text": result["text"],
                "error": result["error"] or ""
            })
            
            if result["text"]:
                preview = result["text"][:100] + "..." if len(result["text"]) > 100 else result["text"]
                logger.info(f"✓ {os.path.basename(key)}: {preview}")
            else:
                logger.warning(f"✗ {os.path.basename(key)}: {result['error']}")
            
            # Periodic flush
            if len(self.results_buffer) >= self.config.append_every_n:
                self._flush_results()
            
            # Memory cleanup
            self.transcription_manager.cleanup_gpu_memory()
            
        except Exception as e:
            tb = traceback.format_exc(limit=2)
            self.results_buffer.append({
                "s3_key": key,
                "filename": os.path.basename(key),
                "transcribed_text": "",
                "error": f"{type(e).__name__}: {e} | {tb}"
            })
            logger.error(f"Failed to process {key}: {e}")
        
        finally:
            # Clean up temp file
            if local_path and os.path.exists(local_path):
                try:
                    os.remove(local_path)
                except Exception:
                    pass
    
    def _flush_results(self):
        """Flush results buffer to CSV"""
        if self.results_buffer:
            self.csv_manager.append_results(self.results_buffer)
            self.results_buffer.clear()


In [8]:
# ======================================== Run main loop ========================================
def main():
    try:
        config = Config()
        pipeline = ProductionPipeline(config)
        pipeline.run()
        return 0
    except KeyboardInterrupt:
        logger.info("Interrupted by user")
        return 130
    except Exception as e:
        logger.error(f"Pipeline failed: {e}")
        logger.error(traceback.format_exc())
        return 1


if __name__ == "__main__":
    result = main()
    if result == 0:
        print("Pipeline completed successfully!")
    else:
        print(f"Pipeline failed with code: {result}")

2025-10-05 20:12:43,667 INFO - Loading ASR: openai/whisper-base (device=cpu, dtype=torch.float32)


Device set to use cpu
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).


2025-10-05 20:12:45,933 INFO - ffmpeg shim created: /home/sagemaker-user/.local/bin/ffmpeg
2025-10-05 20:12:46,175 INFO - Processing 50 files (skipped 0 already done)


Batch 1:   0%|          | 0/8 [00:00<?, ?it/s]

2025-10-05 20:12:51,726 INFO - ✓ common_voice_en_100.mp3:  I admit that I'm an alcoholic.


...

2025-10-05 20:18:07,305 INFO - ✓ common_voice_en_10004.mp3:  The rule has been long because Paris.


Batch 7:  50%|█████     | 1/2 [00:07<00:07,  7.24s/it]

2025-10-05 20:18:12,001 INFO - ✓ common_voice_en_100040.mp3:  The burning fire had been extinguished.


Batch 7: 100%|██████████| 2/2 [00:11<00:00,  5.97s/it]

2025-10-05 20:18:12,332 INFO - Appended 10 rows to ./transcripts_from_prefix.csv
2025-10-05 20:18:12,333 INFO - Pipeline complete!
Pipeline completed successfully!
